In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests, re, os
import pandas as pd
import numpy as np
import time
import datetime
import cv2

## 先抓一個網頁

In [9]:
r = requests.get('https://www.ss.net.tw/paint-176-6236.html')
print(r.status_code)
soup = BeautifulSoup(r.text, 'html.parser')
soup.prettify()

200


'<!DOCTYPE html>\n<html lang="zh-Hant-TW">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="zh" name="content-language"/>\n  <meta content="document" name="resource-type"/>\n  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>\n  <meta content="index,follow" name="robots"/>\n  <meta content="zh_TW" name="language"/>\n  <meta content="Michael．凃" name="author"/>\n  <meta content="【名畫檔案】網路畫廊" name="publisher"/>\n  <meta content="【名畫檔案】" name="company"/>\n  <meta content="ss@ss.net.tw" name="reply-to"/>\n  <meta content="global" name="distribution"/>\n  <meta content="3 days" name="revisit-after"/>\n  <meta content="一個女人的半身像,Buste,femme,Picasso052,男子" name="keywords"/>\n  <meta content="期許從這裡出去的每一幅畫品，都能因我們實事求是的歷程，而包含了知識在裡面。" name="description"/>\n  <meta content="telephone=no,email=no,address=no" name="format-detection"/>\n  <meta content="一個女人的半身像Bustedefemme:男子 - 【名畫檔案】" property="og:title"/>\n  <meta content="painter" property="og:type"/>\n  <me

In [12]:
### 抓資訊 (title, 編號, 作者, 年份, 原作尺寸, 原作材質, 瀏覽人次, 圖片連結)
inf = soup.find_all(id='productsInfo')
inf[0].prettify

<bound method Tag.prettify of <div class="col-md-7 col-lg-6 px-md-0 pt-2 pt-md-0" id="productsInfo"><h1 class="mb-3" data-pid="6236">一個女人的半身像 <small class="d-inline-block">Buste de femme</small></h1><ul class="list-unstyled"><li><label>編　　號</label>Picasso052</li><li class="d-flex"><label class="text-nowrap">作　　者</label><span>畢卡索 <small>Pablo Picasso</small></span></li><li><label>年　　份</label>1938</li><li><label>原作尺寸</label>65.1 x 54 <sub>cm</sub></li><li><label>原作材質</label>油彩．畫布 <small>Oil on Canvas</small></li><li><label>瀏覽人次</label>10688</li></ul><div class="d-flex align-items-center my-3"><a class="btn btn-outline-danger rounded-circle" data-url="set=ss&amp;pID=6236" href="Javascript:;" id="inquiry" title="訂購複製畫"><i class="fas fa-cart-arrow-down fa-lg"></i></a><div class="ml-3"><small class="text-light">置入購物車後可調整所需尺寸</small></div></div><div id="navigator"></div></div>>

In [4]:
### 1. title
title = inf[0].find_all('h1')[0].text #title
title

'一個女人的半身像 Buste de femme'

In [5]:
### 2. 編　　號, 作　　者, 年　　份, 原作尺寸, 原作材質, 瀏覽人次
inf_ = inf[0].find_all('li') #其他資訊
values = []
for i in inf_:
    index = i.find_all('label')[0].text
    if index == '作　　者':
        value = i.find_all('span')[0].text
    elif index == '年　　份':
        value = int(i.contents[1][:4])
    elif index == '原作尺寸':
        value = i.contents[1].split(' x ')
        values.append(float(value[0]))
        values.append(float(value[1]))
        continue
    else:
        value = i.contents[1]
    values.append(value)
values

['Picasso052', '畢卡索 Pablo Picasso', 1938, 65.1, 54.0, '油彩．畫布 ', '10687']

In [163]:
### 3. 抓圖片連結
img_tag = soup.find_all('a', href=re.compile('img'))[0].get('href')
img_url = 'https://www.ss.net.tw/'+img_tag
img_url

'https://www.ss.net.tw/img?o=UGljYXNzbzA1Mi5qcGc='

In [164]:
### 4. 儲存圖片
r_img = requests.get(img_url)
images_dir = 'images02/'
if not os.path.exists(images_dir):
    os.mkdir(images_dir)
with open(images_dir+'image_name.jpg', 'wb') as handler:
    handler.write(r_img.content)

## 爬多個網頁

### 辨識圖片顏色以設定標籤

In [196]:
color_dict = {
    ## hsv範圍: [0-180, 0-255, 0-255]
    'black':[[0,0,0], [180,255,60]],
    'gray':[[0,0,46], [180,30,180]],
    'white':[[0,0,200], [180,30,255]],
    'red':[[0,43,46], [15,255,255]],
    'orange':[[11,50,55], [30,255,255]],
    'yellow':[[22,50,50], [34,255,255]],
    'green':[[35,43,46], [77,255,255]],
    'blue':[[78,43,46], [124,255,255]],
    'purple':[[125,43,46], [155,255,255]]
}
color_df = pd.DataFrame(color_dict, index = ['low', 'high'])

def calcPercentage(msk): 
    
    #計算黑色面積比例是否超過一定面積
    
    #returns the percentage of white in a binary image
    height, width = msk.shape[:2] 
    num_pixels = height * width 
    count_white = cv2.countNonZero(msk) 
    
    ##判斷黑色面積所佔的比例
    percent_white = (1-count_white/num_pixels) * 100
    percent_black = round(percent_white,2)
    return percent_black < 88


def set_color_label(image):
    
    # 設定圖片顏色標籤
    
    labels = []
    # Convert BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    for i in color_df:
        low = np.array(color_df[i].low)
        high = np.array(color_df[i].high)
        # 設定在某一顏色的範圍
        mask = cv2.inRange(hsv, low, high)
        # Bitwise-AND mask and original image
        output = cv2.bitwise_and(image,image, mask= mask)
        
        #紅色特別處理
        maskred = cv2.inRange(hsv, np.array([170, 70, 50]), np.array([180, 255, 255]))
        if i=='red': 
            output2 = cv2.bitwise_and(image, image, mask = maskred)
            output = output+output2
            
        #黑色面積超過一定比例就新增此顏色標籤
        if calcPercentage(mask):
            labels.append(i)
            
    return labels

### 建立 DataFrame

In [197]:
inf_df = pd.DataFrame(
    columns=['id', 'k', 'name','number','author','year','length', 'width',
             'material','view','imglink', 'color_label']
)
df_index = 0
k = 1

### 開始

In [204]:
#is2000 = 0
start = datetime.datetime.now()
while k<=9644:
    try:
        t0 = datetime.datetime.now()
        r = requests.get('https://www.ss.net.tw/paint-177-%s.html'%k)
        if r.status_code != 200:
            print('讀取失敗, k=%s'%k)
            continue
        soup = BeautifulSoup(r.text, 'html.parser')
        inf = soup.find_all(id='productsInfo')
    
        #### 1. 抓 title
        title = inf[0].find_all('h1')[0].text
        
        #### 2. 編　　號, 作　　者, 年　　份, 原作尺寸, 原作材質, 瀏覽人次
        inf_ = inf[0].find_all('li')
        values = [0, k, title]
        for i in inf_:
            index = i.find_all('label')[0].text
            if index == '作　　者':
                value = i.find_all('span')[0].text
            elif index == '年　　份':
                value = int(i.contents[1][:4])
            elif index == '原作尺寸':
                value = i.contents[1].split(' x ')
                values.append(float(value[0]))
                values.append(float(value[1]))
                continue
            else:
                value = i.contents[1]
            values.append(value)
            
        #### 3. 抓圖片網址
        img_tag = soup.find_all('a', href=re.compile('img'))[0].get('href')
        img_url = 'https://www.ss.net.tw/'+img_tag
        values.append(img_url)
        
        #### 4. 下載圖片
        r_img = requests.get(img_url)
        images_dir = 'images03/'
        if not os.path.exists(images_dir):
            os.mkdir(images_dir)
        with open(images_dir+str(k)+'.jpg', 'wb') as handler:
            handler.write(r_img.content)
        
        #### 5. 辨識圖片顏色
        image = cv2.imread(images_dir+str(k)+'.jpg')
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        color_label = set_color_label(image)
        values.append(color_label)
        
        #### 6. 新增到資料到 DataFrame
        inf_df.loc[df_index] = values
        df_index += 1
        
        t1 = datetime.datetime.now()
        dt = (t1-t0).seconds
        print('第 %d 筆資料, 花費 %d 秒, k=%d, 成功'%(df_index, dt, k))

    except Exception as e:
        print('https://www.ss.net.tw/paint-177-%s.html'%k,e)
    k += 1

end = datetime.datetime.now()
times = end-start
print('程式結束，共抓取 %d 筆資料，共花費時間 %d。'%(df_index, times))
    

第 2559 筆資料, 花費 4 秒, k=2642, 成功
第 2560 筆資料, 花費 5 秒, k=2643, 成功
第 2561 筆資料, 花費 5 秒, k=2644, 成功
第 2562 筆資料, 花費 5 秒, k=2645, 成功
第 2563 筆資料, 花費 6 秒, k=2646, 成功
第 2564 筆資料, 花費 4 秒, k=2647, 成功
第 2565 筆資料, 花費 5 秒, k=2648, 成功
第 2566 筆資料, 花費 5 秒, k=2649, 成功
第 2567 筆資料, 花費 6 秒, k=2650, 成功
第 2568 筆資料, 花費 6 秒, k=2651, 成功
第 2569 筆資料, 花費 4 秒, k=2652, 成功
第 2570 筆資料, 花費 5 秒, k=2653, 成功
第 2571 筆資料, 花費 4 秒, k=2654, 成功
第 2572 筆資料, 花費 4 秒, k=2655, 成功
第 2573 筆資料, 花費 3 秒, k=2656, 成功
第 2574 筆資料, 花費 4 秒, k=2657, 成功
第 2575 筆資料, 花費 5 秒, k=2658, 成功
第 2576 筆資料, 花費 4 秒, k=2659, 成功
第 2577 筆資料, 花費 5 秒, k=2660, 成功
第 2578 筆資料, 花費 5 秒, k=2661, 成功
第 2579 筆資料, 花費 5 秒, k=2662, 成功
第 2580 筆資料, 花費 4 秒, k=2663, 成功
第 2581 筆資料, 花費 5 秒, k=2664, 成功
第 2582 筆資料, 花費 5 秒, k=2665, 成功
第 2583 筆資料, 花費 6 秒, k=2666, 成功
第 2584 筆資料, 花費 4 秒, k=2667, 成功
第 2585 筆資料, 花費 5 秒, k=2668, 成功
第 2586 筆資料, 花費 6 秒, k=2669, 成功
第 2587 筆資料, 花費 4 秒, k=2670, 成功
第 2588 筆資料, 花費 4 秒, k=2671, 成功
第 2589 筆資料, 花費 4 秒, k=2672, 成功
第 2590 筆資料, 花費 5 秒, k=2673, 成功
第 2591 筆

TypeError: %d format: a number is required, not datetime.timedelta

### 一開始忘記爬小張的圖片，後來才重新爬一次，再把它加進原本爬好的dataframe

In [216]:
r = requests.get('https://www.ss.net.tw/paint-177-%s.html'%k)

In [274]:
inf_reset_index = inf_df.reset_index()
inf_02 = inf_reset_index.drop('index', axis=1)
#inf_02

In [282]:
#重爬 small image link
#避免出錯，用 while loop爬
small_img = []
i = 0
tsum = 0
while i<=9407:
    t0 = datetime.datetime.now()
    k = inf_02.iloc[i].k
    r = requests.get('https://www.ss.net.tw/paint-177-%s.html'%k)
    soup = BeautifulSoup(r.text, 'html.parser')
    inf = soup.find_all('img', 'lazyload')[0]['data-src']#, _class="lazyload")
    small_img.append(inf)
    t1 = datetime.datetime.now()
    td = (t1-t0).seconds
    tsum += td
    print('i=%d, k=%d 成功, 花費%d秒'%(i, k, td), r.url, inf)
    i+=1
print('結束, 花費%d秒'%tsum)

i=0, k=1 成功, 花費1秒 https://www.ss.net.tw/paint-177-1.html https://www.ss.net.tw/images/product_images/popup_images/AlmaTedema001.jpg
i=1, k=2 成功, 花費1秒 https://www.ss.net.tw/paint-177-2.html https://www.ss.net.tw/images/product_images/popup_images/AlmaTedema002.jpg
i=2, k=3 成功, 花費0秒 https://www.ss.net.tw/paint-177-3.html https://www.ss.net.tw/images/product_images/popup_images/AlmaTedema003.jpg
i=3, k=4 成功, 花費1秒 https://www.ss.net.tw/paint-177-4.html https://www.ss.net.tw/images/product_images/popup_images/AlmaTedema004.jpg
i=4, k=5 成功, 花費1秒 https://www.ss.net.tw/paint-177-5.html https://www.ss.net.tw/images/product_images/popup_images/AlmaTedema005.jpg
i=5, k=6 成功, 花費1秒 https://www.ss.net.tw/paint-177-6.html https://www.ss.net.tw/images/product_images/popup_images/AlmaTedema006.jpg
i=6, k=7 成功, 花費1秒 https://www.ss.net.tw/paint-177-7.html https://www.ss.net.tw/images/product_images/popup_images/AlmaTedema007.jpg
i=7, k=8 成功, 花費1秒 https://www.ss.net.tw/paint-177-8.html https://www.ss.net.

In [287]:
len(small_img)

9408

In [288]:
inf_02.shape

(9408, 12)

In [289]:
inf_03 = inf_02.assign(small_img = small_img)
inf_03

,id,k,name,number,author,year,length,width,material,view,imglink,color_label,small_img
0,0,1,期望 Expectations,AlmaTedema001,亞瑪泰得瑪 Sir Lawrence Alma Tadema,1885,45.0,66.1,油彩．畫布,5246,https://www.ss.net.tw/img?o=QWxtYVRlZGVtYTAwMS...,"[white, blue]",https://www.ss.net.tw/images/product_images/po...
1,0,2,希臘婦女 A Greek Women,AlmaTedema002,亞瑪泰得瑪 Sir Lawrence Alma Tadema,1869,58.5,46.0,油彩．畫板,4074,https://www.ss.net.tw/img?o=QWxtYVRlZGVtYTAwMi...,"[red, orange]",https://www.ss.net.tw/images/product_images/po...
2,0,3,無知的對手 Unconscious Rivals,AlmaTedema003,亞瑪泰得瑪 Sir Lawrence Alma Tadema,1893,45.1,62.8,油彩．畫布,4799,https://www.ss.net.tw/img?o=QWxtYVRlZGVtYTAwMy...,"[red, orange]",https://www.ss.net.tw/images/product_images/po...
3,0,4,最佳位置 A Coign of Vantage,AlmaTedema004,亞瑪泰得瑪 Sir Lawrence Alma Tadema,1895,64.0,44.5,油彩．畫布,3776,https://www.ss.net.tw/img?o=QWxtYVRlZGVtYTAwNC...,"[gray, white, orange, blue]",https://www.ss.net.tw/images/product_images/po...
4,0,5,黑利阿迦八魯斯的玫瑰 The Roses of Heliogabalus,AlmaTedema005,亞瑪泰得瑪 Sir Lawrence Alma Tadema,1888,132.1,213.9,油彩．畫布,2881,https://www.ss.net.tw/img?o=QWxtYVRlZGVtYTAwNS...,"[red, orange]",https://www.ss.net.tw/images/product_images/po...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9403,0,9634,夜窗 Night Windows,Hopper029,霍普 Edward Hopper,1928,73.7,86.4,油彩．畫布,296,https://www.ss.net.tw/img?o=OTYzNF8wMTYzNDAyMj...,"[black, gray, orange]",https://www.ss.net.tw/images/product_images/po...
9404,0,9635,阿特湖古堡一 Schloss Kammer on the Attersee I,Klimt093,克林姆 Gustav Klimt,1908,110.0,110.0,油彩．畫布,326,https://www.ss.net.tw/img?o=OTYzNV8wMTYzNDQ3Nj...,"[orange, green, blue]",https://www.ss.net.tw/images/product_images/po...
9405,0,9642,聖伊格內修斯的神化 Apotheosis of Saint Ignatius,Pozzo001,安德烈亞．波佐 Andrea Pozzo,1674,1700.0,3200.0,壁畫,132,https://www.ss.net.tw/img?o=OTY0Ml8wMTYzNTg2Mj...,"[gray, red, orange, yellow]",https://www.ss.net.tw/images/product_images/po...
9406,0,9643,蘇爾門肖像 Marten Soolmans,Rembrandt139,林布蘭特 Rembrandt HarMenszoon van Rijn,1634,207.0,132.5,油彩．畫布,294,https://www.ss.net.tw/img?o=OTY0M18wMTYzNjEwOD...,"[black, gray, red, orange]",https://www.ss.net.tw/images/product_images/po...


In [290]:
# 輸出 data
inf_03.to_csv('data03_1.csv', index = False, encoding='utf-8')